In [29]:
import pandas as pd 
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from collections import Counter
from deep_translator import GoogleTranslator
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [3]:
df = pd.read_excel("Data for Task 2.xlsx")

In [4]:
df.head(5)

,Primary Key,Order No,Segment Number,Order Date,Manufacturer,Model,Product Category,Model Year,Serial Number,Meter 1 Reading,Complaint,Cause,Correction,Failure Condition - Failure Component,Fix Condition - Fix Component
0,SO0005588-1,SO0005588,1,2022-04-30,PASEIH,6780,APPL,0,YFT042399,2531.0999,"No cab heat, temp gauge dont get to operating ...",NaN,"Als ich das Gerät in die Werkstatt fuhr, stieg...","No Heat - Cab, Not Achieving - Gauge",No Component Mentioned - Added
1,SO0005907-1,SO0005907,1,2022-04-30,PASEIH,6780,APPL,0,YFT042399,2531.0999,"No cab heat, temp gauge dont get to operating ...",NaN,"Als ich das Gerät in die Werkstatt fuhr, stieg...","No Heat - Cab, Not Achieving - Gauge",No Component Mentioned - Added
2,SO0006100-1,SO0006100,1,2022-04-30,PASEIH,6780,APPL,0,YFT042399,2531.0999,"No cab heat, temp gauge dont get to operating ...",NaN,"Als ich das Gerät in die Werkstatt fuhr, stieg...",Not Charging - Alternator,No Component Mentioned - No Component Mentioned
3,SO0006642-1,SO0006642,1,2022-04-30,PASEIH,6780,APPL,0,YFT042399,2531.0999,"No cab heat, temp gauge dont get to operating ...",NaN,"Als ich das Gerät in die Werkstatt fuhr, stieg...",Faulty - Fan,"Tensioner - Removed, Crankshaft Pulley - Cleaned"
4,SO0018457-1,SO0018457,1,2022-04-30,PASEIH,6780,APPL,0,YFT042399,2531.0999,"No cab heat, temp gauge dont get to operating ...",NaN,"Als ich das Gerät in die Werkstatt fuhr, stieg...",Oil Loss - Not Mentioned,No Component Mentioned - No Component Mentioned


In [5]:
df.isnull().sum()### checking NULL values found in Cause and Correction

Primary Key                                0
Order No                                   0
Segment Number                             0
Order Date                                 0
Manufacturer                               0
Model                                      0
Product Category                           0
Model Year                                 0
Serial Number                              0
Meter 1 Reading                            0
Complaint                                  0
Cause                                    294
Correction                                25
Failure Condition - Failure Component      0
Fix Condition - Fix Component              0
dtype: int64

In [6]:
df.shape

(500, 15)

In [7]:
df = df.drop(columns=['Cause'])### Many Null values in these column , so i dropped

In [8]:
df.isnull().sum()

Primary Key                               0
Order No                                  0
Segment Number                            0
Order Date                                0
Manufacturer                              0
Model                                     0
Product Category                          0
Model Year                                0
Serial Number                             0
Meter 1 Reading                           0
Complaint                                 0
Correction                               25
Failure Condition - Failure Component     0
Fix Condition - Fix Component             0
dtype: int64

In [9]:
df = df.dropna(subset=['Correction'])### I dropped the rows which has NULL values

In [10]:
df.isnull().sum()

Primary Key                              0
Order No                                 0
Segment Number                           0
Order Date                               0
Manufacturer                             0
Model                                    0
Product Category                         0
Model Year                               0
Serial Number                            0
Meter 1 Reading                          0
Complaint                                0
Correction                               0
Failure Condition - Failure Component    0
Fix Condition - Fix Component            0
dtype: int64

In [42]:
#### Here I translate the complaint column which is in different language.
# Define the column to translate
text_col = 'Complaint'

# Function to translate text if not null
def translate_text(text):
    if pd.isna(text) or text.strip() == "":
        return text
    try:
        return GoogleTranslator(source='auto', target='en').translate(text)
    except Exception as e:
        print(f"Error translating: {text[:50]}... | {e}")
        return text

# Apply translation
df['Complaint'] = df[text_col].apply(translate_text)

# View some translated results
print(df[['Complaint']].head())

                                           Complaint
0  No cab heat, temp gauge dont get to operating ...
1  No cab heat, temp gauge dont get to operating ...
2  No cab heat, temp gauge dont get to operating ...
3  No cab heat, temp gauge dont get to operating ...
4  No cab heat, temp gauge dont get to operating ...


In [26]:
###Here I translate the correction column which is in different language.

# Define the column to translate
text_col = 'Correction'

# Function to translate text if not null
def translate_text(text):
    if pd.isna(text) or text.strip() == "":
        return text
    try:
        return GoogleTranslator(source='auto', target='en').translate(text)
    except Exception as e:
        print(f"Error translating: {text[:50]}... | {e}")
        return text

# Apply translation
df['Correction_Translated'] = df[text_col].apply(translate_text)

# View some translated results
print(df[['Correction', 'Correction_Translated']].head())


                                          Correction  \
0  Als ich das Gerät in die Werkstatt fuhr, stieg...   
1  Als ich das Gerät in die Werkstatt fuhr, stieg...   
2  Als ich das Gerät in die Werkstatt fuhr, stieg...   
3  Als ich das Gerät in die Werkstatt fuhr, stieg...   
4  Als ich das Gerät in die Werkstatt fuhr, stieg...   

                               Correction_Translated  
0  When I drove the device into the workshop, the...  
1  When I drove the device into the workshop, the...  
2  When I drove the device into the workshop, the...  
3  When I drove the device into the workshop, the...  
4  When I drove the device into the workshop, the...  


In [12]:
df1 = pd.read_csv('task1.csv')

In [48]:
df

,Primary Key,Order No,Segment Number,Order Date,Manufacturer,Model,Product Category,Model Year,Serial Number,Meter 1 Reading,Complaint,Correction,Failure Condition - Failure Component,Fix Condition - Fix Component,Correction_Translated
0,SO0005588-1,SO0005588,1,2022-04-30,PASEIH,6780,APPL,0,YFT042399,2531.0999,"No cab heat, temp gauge dont get to operating ...","Als ich das Gerät in die Werkstatt fuhr, stieg...","No Heat - Cab, Not Achieving - Gauge",No Component Mentioned - Added,"When I drove the device into the workshop, the..."
1,SO0005907-1,SO0005907,1,2022-04-30,PASEIH,6780,APPL,0,YFT042399,2531.0999,"No cab heat, temp gauge dont get to operating ...","Als ich das Gerät in die Werkstatt fuhr, stieg...","No Heat - Cab, Not Achieving - Gauge",No Component Mentioned - Added,"When I drove the device into the workshop, the..."
2,SO0006100-1,SO0006100,1,2022-04-30,PASEIH,6780,APPL,0,YFT042399,2531.0999,"No cab heat, temp gauge dont get to operating ...","Als ich das Gerät in die Werkstatt fuhr, stieg...",Not Charging - Alternator,No Component Mentioned - No Component Mentioned,"When I drove the device into the workshop, the..."
3,SO0006642-1,SO0006642,1,2022-04-30,PASEIH,6780,APPL,0,YFT042399,2531.0999,"No cab heat, temp gauge dont get to operating ...","Als ich das Gerät in die Werkstatt fuhr, stieg...",Faulty - Fan,"Tensioner - Removed, Crankshaft Pulley - Cleaned","When I drove the device into the workshop, the..."
4,SO0018457-1,SO0018457,1,2022-04-30,PASEIH,6780,APPL,0,YFT042399,2531.0999,"No cab heat, temp gauge dont get to operating ...","Als ich das Gerät in die Werkstatt fuhr, stieg...",Oil Loss - Not Mentioned,No Component Mentioned - No Component Mentioned,"When I drove the device into the workshop, the..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,SO0060999-1,SO0060999,1,2024-02-19,PASEIH,6780,FLOATE,2019,JFGYF44BTJT048253,2662.0000,T/S Transmission,Vorherige Anmerkungen zur Excel-Tabelle vom Mo...,Hydraulic Issue - Sensor,Oil Temperature Sensor - Replaced,Previous comments on the Excel table on Monday...
496,SO0060999-2,SO0060999,2,2024-02-19,PASEIH,9991,SPRAYS,0,JFGYCC89CPT057043,218.0000,Hyd oil leaking from Hydraulic Block under the...,"2.19.24 – An der Maschine angekommen, inspizie...","Rate Issue - Machine, Chain Issue - Machine, F...","Rear Sensor - Replaced, Rear Sensor - Recalibr...","2.19.24-Arrived, inspected and found on the ma..."
497,SO0061006-1,SO0061006,1,2024-02-19,PASEIH,9991,SPRAYS,0,JFGYCC89CPT057043,218.0000,Hyd oil leaking from Hydraulic Block under the...,"2.19.24 – An der Maschine angekommen, inspizie...",Not Folding - Boom,"Sensor - Replaced, Boom - Recalibrated, Softwa...","2.19.24-Arrived, inspected and found on the ma..."
498,SO0061019-1,SO0061019,1,2024-02-19,PASEIH,9991,SPRAYS,0,JFGYCC89CPT057043,218.0000,STEERING COLUMN TILT/TELESCOPE FUNCTION IS FRE...,19.02.24 – AN DER MASCHINE ANGEKOMMEN UND ÜBER...,Error Code - Unit,No Component Mentioned - No Component Mentioned,02/19/24 - Arrived on the machine and checked ...


In [64]:
###here i merged two data sets using outerjoin , because i couldn't found primary key in set1 so i used VIN as Primary key.

# Ensure both columns are string type
df1['VIN'] = df1['VIN'].astype(str)
df['Primary Key'] = df['Primary Key'].astype(str)

# Outer join on VIN and Primary Key
merged_df = pd.merge(
    df1,
    df,
    left_on='VIN',
    right_on='Primary Key',
    how='outer',
    suffixes=('_task1', '_task2')
)

# Inspect results
print(f"Merged shape: {merged_df.shape}")
merged_df.head()


Merged shape: (563, 70)


,VIN,TRANSACTION_ID,CORRECTION_VERBATIM,CUSTOMER_VERBATIM,REPAIR_DATE,CAUSAL_PART_NM,GLOBAL_LABOR_CODE_DESCRIPTION,PLATFORM,BODY_STYLE,VPPC,...,Model,Product Category,Model Year,Serial Number,Meter 1 Reading,Complaint,Correction,Failure Condition - Failure Component,Fix Condition - Fix Component,Correction_Translated
0,1H1FF1R79N0134651,13034.0,replaced steering wheel cover loose inside,cs steering wheel cover loose moving,2024-01-29,wheel asmstrg jet black,Steering Wheel Replacement,Alpha,2 Door Coupe,A1BC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1H1JD5SB0L4142407,13039.0,replace steering wheel sop ordered ro 412613,cs steering wheel pealing separatinginstall so...,2024-01-25,wheel asmstrg jet black,Steering Wheel Replacement,Global Gamma Vehicles,4 Door Sedan,G1SC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1H6DC5RK2L0132064,13036.0,corr 0130 replace steering wheel,sop per customer leather wrap back steering wh...,2024-01-31,wheel asmstrg jet black,Steering Wheel Replacement,Luxury Car-2,4 Door Sedan,A2SL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1H6DS5RK6S0127345,13025.0,replaced heated steering wheel,customer states heated steering wheel isnt wor...,2024-01-10,wheel asmstrg black,Steering Wheel Replacement,Luxury Car-3,4 Door Sedan,A2LL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1HC4WLE78RF260518,13045.0,removed steering wheel disassembled found scre...,customer states clicking type noise coming ste...,2024-02-07,wheel asmstrg jet black,Steering Wheel Replacement,Full-Size Trucks,Crew Cab,T1CCH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
merged_df.to_csv("Final.csv", index =False)